In [1]:
#Prerequisites

import sys
!{sys.executable} -m pip install --user networkx
!{sys.executable} -m pip install --user numpy
!{sys.executable} -m pip install --user pandas

import networkx as nx
import numpy as np
import pandas as pd
import os

from pandas import DataFrame

In [5]:
# Read in Source File - NB this must match the schema requirements
# read in data
initial_report_SF = pd.read_csv("~/Documents/Trellis/Network_Analysis_Tableau/data/initial-report-SF2.csv")
CodeType = 'latin1' # https://docs.python.org/3/library/codecs.html#standard-encodings
print(initial_report_SF.tail())

                  Appointment Name  Case Number Case Detail Name  \
119972  4/1/2024 - Samuel Martinez    2658465.0       CD-1521038   
119973       4/1/2024 - Jiya Singh    2658471.0       CD-1521041   
119974   4/1/2024 - Madison Rogers    2658472.0       CD-1521042   
119975       4/1/2024 - Will Jones    2658479.0       CD-1521045   
119976     4/1/2024 - Ruby Lenertz    2658482.0       CD-1521046   

        Owner: Owner ID  Contact EmplID          Case ID Appointment Date  \
119972  0053n000007NsoN      23871273.0  5006R00001zCVeW           4/1/24   
119973  0052S00000AS2RV      23867493.0  5006R00001zCVfZ           4/1/24   
119974  005f4000004dmIP             NaN  5006R00001zCVfe           4/1/24   
119975  005f4000004dmIN      23737588.0  5006R00001zCVgr           4/1/24   
119976  0052S000009vjRh      23716506.0  5006R00001zCVhB           4/1/24   

              Advisor Name  
119972     Ursula Basinger  
119973        Meaghan Shaw  
119974  Stephanie Thiltges  
119975     D

/var/folders/wr/p82k0vm11j17r80bp7wf3swc0000gs/T/ipykernel_1761/3265630707.py:3: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  initial_report_SF = pd.read_csv("~/Documents/Trellis/Network_Analysis_Tableau/data/initial-report-SF2.csv")


In [6]:
# Get variable names (column names) of the DataFrame
variable_names = initial_report_SF.columns

print(variable_names)

Index(['Appointment Name', 'Case Number', 'Case Detail Name',
       'Owner: Owner ID', 'Contact EmplID', 'Case ID', 'Appointment Date',
       'Advisor Name'],
      dtype='object')


In [7]:
# Filter out rows where `Case ID` or `Contact EmplID` is NA or empty
filtered_df = initial_report_SF.dropna(subset=['Case Number', 'Case ID', 'Contact EmplID'])
#initial_report_SF = initial_report_SF[(~initial_report_SF['Case ID'].isna()) | (initial_report_SF['Case ID'] != "")]
#initial_report_SF = initial_report_SF[(~initial_report_SF['Contact EmplID'].isna()) | (initial_report_SF['Contact EmplID'] != "")]
print(filtered_df.head())

                 Appointment Name  Case Number Case Detail Name  \
66441  7/3/2023 - Helena Lassalle    2249069.0       CD-1405953   
66442     7/3/2023 - Jp Cervantes    2249145.0       CD-1405955   
66443       7/3/2023 - Bella Bush    2249152.0       CD-1405956   
66444   7/3/2023 - Stelyne S Claw    2249181.0       CD-1405965   
66445   7/3/2023 - Levin Mullaney    2249215.0       CD-1405978   

       Owner: Owner ID  Contact EmplID          Case ID Appointment Date  \
66441  0052S000009vjSY      23847450.0  5003n00002iJFYc           7/3/23   
66442  0052S000009vIbI      23641362.0  5003n00002iJGhj           7/3/23   
66443  0053n000007s0nJ      23828411.0  5003n00002iJG2a           7/3/23   
66444  0052S000009vjS4      23531788.0  5003n00002iJGtU           7/3/23   
66445  0052S000009vjSB      23883597.0  5003n00002iJHAu           7/3/23   

            Advisor Name  
66441  Stephanie Maxwell  
66442       Monique Pena  
66443          Rani Metz  
66444    Josette Miranda  
66445

In [9]:
# Convert 'Appointment Date' column to datetime
filtered_df['AppointmentDate'] = pd.to_datetime(filtered_df['Appointment Date'])

# Calculate today's date
today = pd.to_datetime('today').normalize()
filtered_df['today'] = today

# Calculate days since 'AppointmentDate'
filtered_df['days_since'] = (filtered_df['today'] - filtered_df['AppointmentDate']).dt.days

# filter dates to 2024
#filtered_df[(filtered_df['date']>datetime.date(2024,1,1)) & (filtered_df['date']<datetime.date(2024,5,5))]
filtered_df = filtered_df[(filtered_df['AppointmentDate'] > "2024-01-01") & (filtered_df['AppointmentDate'] < "2024-07-01")]
# Display the updated DataFrame
print(filtered_df.head())

                    Appointment Name  Case Number Case Detail Name  \
70365        4/21/2024 - Alyssa Kane    2290129.0       CD-1415252   
101304       1/2/2024 - Maryn Murphy    2513167.0       CD-1476553   
101305        1/2/2024 - Sthuthi Das    2513173.0       CD-1476555   
101306  1/2/2024 - Annettia Campbell    2513175.0       CD-1476557   
101307     1/2/2024 - Jaz Youngblood    2513269.0       CD-1476573   

        Owner: Owner ID  Contact EmplID          Case ID Appointment Date  \
70365   0053n000008Xk4r      23672987.0  5003n00002jVthI          4/21/24   
101304  0052S00000ASqE1      23818566.0  5006R00001yH5qN           1/2/24   
101305  0053n000007kjY3      23796945.0  5006R00001yH5r6           1/2/24   
101306  0052S000009vjRl      23875861.0  5006R00001yH5rf           1/2/24   
101307  0053n0000051PtI      23509748.0  5006R00001yH6Hi           1/2/24   

                       Advisor Name AppointmentDate      today  days_since  
70365                Karine Michael    

In [15]:
# create new emplID's
# Extract unique IDs
unique_ids = filtered_df['Contact EmplID'].unique()

# Generate new IDs (random IDs)
np.random.seed(53013)  # for reproducibility
new_ids = np.random.randint(1000, 9999, len(unique_ids))

# Create a dictionary to map old IDs to new IDs
id_mapping = dict(zip(unique_ids, new_ids))

id_mapping
#filtered_df_newID=filtered_df.append(id_mapping,ignore_index=True)


{23672987.0: 4852,
 23818566.0: 6239,
 23796945.0: 7151,
 23875861.0: 7072,
 23509748.0: 7176,
 23708015.0: 3311,
 23557396.0: 3872,
 23580873.0: 6729,
 23744290.0: 5275,
 23748665.0: 5000,
 23707178.0: 8673,
 23879922.0: 6469,
 23645343.0: 9728,
 23473494.0: 8573,
 23925430.0: 8143,
 23575398.0: 7141,
 23876489.0: 2759,
 23592110.0: 3115,
 23475907.0: 8737,
 23702487.0: 8372,
 23929719.0: 2847,
 23736988.0: 1218,
 23836841.0: 9547,
 23842988.0: 7306,
 23816208.0: 5778,
 23560430.0: 7210,
 23794854.0: 6122,
 23729175.0: 6612,
 23577403.0: 6822,
 23792688.0: 1997,
 23748827.0: 5922,
 23810779.0: 3198,
 23819604.0: 6411,
 23698132.0: 7200,
 23655773.0: 2188,
 23761347.0: 5157,
 23701538.0: 5230,
 23307169.0: 8688,
 23618182.0: 6470,
 23709002.0: 8624,
 23780971.0: 7798,
 23811840.0: 3182,
 23779694.0: 3280,
 909238.0: 3557,
 23858309.0: 3853,
 23744570.0: 2775,
 23784821.0: 8606,
 23828792.0: 1239,
 23741977.0: 8509,
 23659409.0: 7826,
 23751501.0: 9616,
 23860337.0: 6143,
 23883049.0: 1

In [16]:
# Get variable names (column names) of the DataFrame
variable_names = filtered_df.columns

print(variable_names)

Index(['Appointment Name', 'Case Number', 'Case Detail Name',
       'Owner: Owner ID', 'Contact EmplID', 'Case ID', 'Appointment Date',
       'Advisor Name', 'AppointmentDate', 'today', 'days_since'],
      dtype='object')


In [17]:
#filtered_df['Contact EmplID'].map(id_mapping)  
filtered_df['New_ID'] = filtered_df['Contact EmplID'].map(id_mapping)

In [18]:
print(filtered_df.head())

                    Appointment Name  Case Number Case Detail Name  \
70365        4/21/2024 - Alyssa Kane    2290129.0       CD-1415252   
101304       1/2/2024 - Maryn Murphy    2513167.0       CD-1476553   
101305        1/2/2024 - Sthuthi Das    2513173.0       CD-1476555   
101306  1/2/2024 - Annettia Campbell    2513175.0       CD-1476557   
101307     1/2/2024 - Jaz Youngblood    2513269.0       CD-1476573   

        Owner: Owner ID  Contact EmplID          Case ID Appointment Date  \
70365   0053n000008Xk4r      23672987.0  5003n00002jVthI          4/21/24   
101304  0052S00000ASqE1      23818566.0  5006R00001yH5qN           1/2/24   
101305  0053n000007kjY3      23796945.0  5006R00001yH5r6           1/2/24   
101306  0052S000009vjRl      23875861.0  5006R00001yH5rf           1/2/24   
101307  0053n0000051PtI      23509748.0  5006R00001yH6Hi           1/2/24   

                       Advisor Name AppointmentDate      today  days_since  \
70365                Karine Michael   

In [19]:
# create a new column to extract the names
# Extract just the names
filtered_df['Student Name'] = filtered_df['Appointment Name'].str.split(' - ', expand=True)[1]

# Display the updated DataFrame
print(filtered_df.head())

                    Appointment Name  Case Number Case Detail Name  \
70365        4/21/2024 - Alyssa Kane    2290129.0       CD-1415252   
101304       1/2/2024 - Maryn Murphy    2513167.0       CD-1476553   
101305        1/2/2024 - Sthuthi Das    2513173.0       CD-1476555   
101306  1/2/2024 - Annettia Campbell    2513175.0       CD-1476557   
101307     1/2/2024 - Jaz Youngblood    2513269.0       CD-1476573   

        Owner: Owner ID  Contact EmplID          Case ID Appointment Date  \
70365   0053n000008Xk4r      23672987.0  5003n00002jVthI          4/21/24   
101304  0052S00000ASqE1      23818566.0  5006R00001yH5qN           1/2/24   
101305  0053n000007kjY3      23796945.0  5006R00001yH5r6           1/2/24   
101306  0052S000009vjRl      23875861.0  5006R00001yH5rf           1/2/24   
101307  0053n0000051PtI      23509748.0  5006R00001yH6Hi           1/2/24   

                       Advisor Name AppointmentDate      today  days_since  \
70365                Karine Michael   

In [20]:
# add an index column

filtered_df['index'] = range(1, len(filtered_df) + 1)
#filtered_df['index'] = filtered_df.index

# Display the updated DataFrame
#print(filtered_df.head())

In [27]:
# create a source column
filtered_df['Source'] = filtered_df.groupby(['Advisor Name']).ngroup()

# Display the updated DataFrame
print(filtered_df.head())

                    Appointment Name  Case Number Case Detail Name  \
70365        4/21/2024 - Alyssa Kane    2290129.0       CD-1415252   
101304       1/2/2024 - Maryn Murphy    2513167.0       CD-1476553   
101305        1/2/2024 - Sthuthi Das    2513173.0       CD-1476555   
101306  1/2/2024 - Annettia Campbell    2513175.0       CD-1476557   
101307     1/2/2024 - Jaz Youngblood    2513269.0       CD-1476573   

               owner_id  Contact EmplID          Case ID Appointment Date  \
70365   0053n000008Xk4r      23672987.0  5003n00002jVthI          4/21/24   
101304  0052S00000ASqE1      23818566.0  5006R00001yH5qN           1/2/24   
101305  0053n000007kjY3      23796945.0  5006R00001yH5r6           1/2/24   
101306  0052S000009vjRl      23875861.0  5006R00001yH5rf           1/2/24   
101307  0053n0000051PtI      23509748.0  5006R00001yH6Hi           1/2/24   

                       Advisor Name AppointmentDate      today  days_since  \
70365                Karine Michael   

In [28]:
filtered_df = filtered_df.rename(columns={'Owner: Owner ID': 'owner_id', 'New_ID': 'Target'})

In [29]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18224 entries, 70365 to 119976
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Appointment Name  18224 non-null  object        
 1   Case Number       18224 non-null  float64       
 2   Case Detail Name  18148 non-null  object        
 3   owner_id          18224 non-null  object        
 4   Contact EmplID    18224 non-null  float64       
 5   Case ID           18224 non-null  object        
 6   Appointment Date  18224 non-null  object        
 7   Advisor Name      18224 non-null  object        
 8   AppointmentDate   18224 non-null  datetime64[ns]
 9   today             18224 non-null  datetime64[ns]
 10  days_since        18224 non-null  int64         
 11  Target            18224 non-null  object        
 12  Student Name      18224 non-null  object        
 13  index             18224 non-null  int64         
 14  Source           

In [30]:
filtered_df['Source'] = filtered_df['Source'].map(str)
filtered_df['Target'] = filtered_df['Target'].map(str)

In [31]:
# Add directions to d3
filtered_df['Direction'] = filtered_df['Source'] + ' -> ' + filtered_df['Target']

In [32]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18224 entries, 70365 to 119976
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Appointment Name  18224 non-null  object        
 1   Case Number       18224 non-null  float64       
 2   Case Detail Name  18148 non-null  object        
 3   owner_id          18224 non-null  object        
 4   Contact EmplID    18224 non-null  float64       
 5   Case ID           18224 non-null  object        
 6   Appointment Date  18224 non-null  object        
 7   Advisor Name      18224 non-null  object        
 8   AppointmentDate   18224 non-null  datetime64[ns]
 9   today             18224 non-null  datetime64[ns]
 10  days_since        18224 non-null  int64         
 11  Target            18224 non-null  object        
 12  Student Name      18224 non-null  object        
 13  index             18224 non-null  int64         
 14  Source           

In [33]:
# FOLLOWING KNIPPENBERG

# Save the DataFrame to a CSV file

filtered_df.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/py-filtered_df.csv', index=False)  # Set index=False to exclude the index from the CSV file


In [34]:
reduced_df = filtered_df.sample(n=300)

In [35]:
arr_SrcTgt = np.array(reduced_df[['Source','Target']])

In [36]:
# Create Network Graph Coordinates...
Q = nx.Graph()
Q.add_edges_from(arr_SrcTgt)
dict_Coords = nx.spring_layout(Q) 

In [37]:
# Create Graph Coordinates File...
df_Raw_Coords = DataFrame(dict_Coords)
df_Raw_Coords = df_Raw_Coords.T
df_Raw_Coords.columns = ['X','Y']
df_Raw_Coords.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/CoordsFile.csv', index_label='NodeName')
print(df_Raw_Coords)

             X         Y
192   0.254707 -0.195603
8804  0.253412 -0.168765
68   -0.557782 -0.568223
2425 -0.574840 -0.563152
27    0.104626  0.892847
...        ...       ...
153  -1.000000  0.165893
3652 -0.962308  0.159191
6641  0.664977 -0.091619
7919 -0.496332  0.426908
5959  0.672247 -0.111974

[419 rows x 2 columns]


In [38]:
# Create Bridge File... 
# Tableau Code: IF [Src-Tgt]/2 = ROUND([Src-Tgt]/2) THEN 'Source' ELSE 'Target' END
arr_SrcTgt2 = arr_SrcTgt.reshape(1,(len(arr_SrcTgt)*2)) 
arr_SrcTgt2 = arr_SrcTgt2.reshape(-1) 
df_SrcTgt = DataFrame(arr_SrcTgt2,columns=['NodeName']) 
arr_Index = []
for i in range(1,(len(arr_SrcTgt)+1)):
    arr_Index.append(i)
    arr_Index.append(i)
df_SrcTgt['c_Index'] = arr_Index 

df_SrcTgt.to_csv('~/Documents/Trellis/Network_Analysis_Tableau/data/generated_from_knipp/BridgeFile.csv',index_label='Src-Tgt')
#print(df_ScrTgt.head())

In [39]:
print('Run Completed Successfully')

Run Completed Successfully
